In [1]:
from importlib import reload
import numpy as np
import time

from ics.cobraCharmer import pfi as pfiControl
reload(pfiControl)

<module 'ics.cobraCharmer.pfi' from '/home/cwen/mhs/devel/ics_cobraCharmer/python/ics/cobraCharmer/pfi.py'>

In [2]:
mod1Cobras = pfiControl.PFI.allocateCobraRange(range(1,2))
allCobras = mod1Cobras
oneCobra = pfiControl.PFI.allocateCobraList([(1,1)])
twoCobras = pfiControl.PFI.allocateCobraList([(1,1), (1,2)])

# partition module 1 cobras into non-interfering sets
moduleCobras = {}
for group in 1,2,3:
    cm = range(group,58,3)
    mod = [1]*len(cm)
    moduleCobras[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
group1Cobras = moduleCobras[1]
group2Cobras = moduleCobras[2]
group3Cobras = moduleCobras[3]

# partition module 1 cobras into odd and even sets
moduleCobras2 = {}
for group in 1,2:
    cm = range(group,58,2)
    mod = [1]*len(cm)
    moduleCobras2[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
oddCobras = moduleCobras2[1]
evenCobras = moduleCobras2[2]

In [3]:
#pfi = pfiControl.PFI(fpgaHost='128.149.77.24') #'fpga' for real device.
pfi = pfiControl.PFI(fpgaHost='localhost', doLoadModel=False)
pfi.loadModel('../xml/2018_03_26_final_xml.xml')
pfi.setFreq(allCobras)

In [4]:
# Calculate up/down angles
oddMoves = pfi.thetaToLocal(oddCobras, [np.deg2rad(135)]*len(oddCobras))
oddMoves[oddMoves>1.85*np.pi] = 0

evenMoves = pfi.thetaToLocal(evenCobras, [np.deg2rad(315)]*len(evenCobras))
evenMoves[evenMoves>1.85*np.pi] = 0

allMoves = np.zeros(57)
allMoves[::2] = oddMoves
allMoves[1::2] = evenMoves

def moveCobra(c, theta, phi):
    pfi.moveSteps([allCobras[c-1]], np.zeros(1)+theta, np.zeros(1)+phi)

def moveCobras(cs, theta, phi):
    cobs = []
    for c in cs:
        cobs.append(allCobras[c-1])
    pfi.moveSteps(cobs, np.array(theta), np.array(phi))

In [5]:
# The updated steps for moving cobras to up/down positions
allSteps = np.array([3200., 4300,    0., 4800., 1400,
 6000.,  1400., 3267.39039139, 2934.04991828, 0.,
 1440.0769006,  1108.36510939, 2447.07789072, 3300., 6000.,
   22.40497809,  233.60670712, 2846.80763061,  717.26507015, 1432.59642308,
  263.22585701, 1200.,  135.92568801, 4880.22316911,    0.,
 1700.,  411.15612259, 4190.63271838,  309.95326555,  937.61978123,
 3527.60958129, 1882.10623443,  667.63393703, 2331.33049693, 1008.7811692,
    0.,         3430.76739972, 2833.79313889,  836.87837639, 4200.,
 1389.91444697, 2214.89547244, 3659.6670764,  900., 3624.95167338,
 3200.,    0.,         4000.,  707.3192973,  2546.98893839,
 1919.88595392, 1753.70977189, 3894.01513043, 3600., 900.,
 1847.58659938,  833.76207202])

In [6]:
# move a few cobras to outward position
cs = [28,31,38,41,46]
cind = np.array(cs) - 1
cnt = len(cind)

for i in range(3):
    # Home theta for multiple cobras
    moveCobras(cs, [-15000] * cnt, [0] * cnt)

    # Move multiple cobras
    moveCobras(cs, allSteps[cind], [0] * cnt)

    # sleep for three second
    time.sleep(3.0)

In [7]:
# move all cobras to outward position
for i in range(3):
    # Home theta for all cobras
    pfi.moveSteps(allCobras, np.zeros(57)-15000, np.zeros(57))

    # Move all cobras to up/down positions
    pfi.moveSteps(allCobras, allSteps, np.zeros(57))

    # sleep for three second
    time.sleep(3.0)

In [8]:
# Home theta
pfi.homeTheta(allCobras, nsteps=15000, dir='ccw')

In [9]:
# Home phi in safe way
pfi.homePhiSafe(allCobras, nsteps=5000, dir='ccw')

In [10]:
# Home phi
pfi.homePhi(allCobras, dir='ccw')

In [11]:
# Move one cobra
moveCobra(4, 4800, 0)

In [12]:
# Move multiple cobras
moveCobras((28, 46), (4800, 5200), (20, -100))

In [13]:
# Move all cobras to up/down positions, the steps are calculated from motor maps
pfi.moveThetaPhi(allCobras, allMoves, np.zeros(57))

thetaSteps:  [1946.37017013 4944.10561599    0.         3612.31363426 1616.55721876
 5737.7411516  2794.76809696 3267.39039139 2934.04991828 4822.8620753
 1440.0769006  1108.36510939 2447.07789072 4433.48606627 5314.14202023
   22.40497809  233.60670712 2846.80763061  717.26507015 1432.59642308
  263.22585701 1246.65632275  135.92568801 4880.22316911    0.
 1932.27263422  411.15612259 4190.63271838  309.95326555  937.61978123
 3527.60958129 1882.10623443  667.63393703 2331.33049693 1008.7811692
    0.         3430.76739972 2833.79313889  836.87837639 5566.04318772
 1389.91444697 2214.89547244 3659.6670764  1083.38998761 3624.95167338
 3660.94310085    0.         2806.92323943  707.3192973  2546.98893839
 1919.88595392 1753.70977189 3894.01513043 6145.93989201 1278.54545365
 1847.58659938  833.76207202]
phiSteps:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# The angles to up/down positions
np.rad2deg(allMoves)

In [ ]:
# Cobra motor map parameters
print(pfi.calibModel.motorFreq1, pfi.calibModel.motorFreq2)
print(pfi.calibModel.motorOntimeFwd1, pfi.calibModel.motorOntimeFwd2)
print(pfi.calibModel.tht0, pfi.calibModel.tht1)

In [16]:
# Script to measure cobra centers, arm lengths, motor maps
steps = 500
zeros = np.zeros(19)
for g, cobraGroup in enumerate(moduleCobras.values()):
    for i in range(10):
        pfi.moveSteps(cobraGroup, zeros, zeros+steps)
        time.sleep(0.5)
    time.sleep(5)
    pfi.moveSteps(cobraGroup, zeros-15000, zeros)
    time.sleep(5)
    for i in range(30):
        pfi.moveSteps(cobraGroup, zeros+steps, zeros)
        time.sleep(0.5)
    time.sleep(5)
    for i in range(30):
        pfi.moveSteps(cobraGroup, zeros-steps, zeros)
        time.sleep(0.5)
    time.sleep(5)
    pfi.moveSteps(cobraGroup, allSteps[g::3], np.zeros(19))
#    pfi.moveThetaPhi(cobraGroup, allMoves[g::3], np.zeros(19))
    time.sleep(5)
    for i in range(10):
        pfi.moveSteps(cobraGroup, zeros, zeros-steps)
        time.sleep(0.5)
    time.sleep(5)